# Анализ коммерческих показателей

In [ ]:
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline


URL = 'https://code.s3.yandex.net/datasets/'

## Метрики и воронки

Процесс продаж: потребитель -(привлечение)-> посетитель -(продажа)-> покупатель .

### Конверсия

Конверсия — это доля людей, перешедших из одного состояния в другое.

### Воронка действий

Воронка действий - это схема, в которой ожидаемые (нужные для коммерции) действия пользователя расставлены так, что каждое действие - это переход из этапа в этап.

"Закрытие сделки" - это проход всех шагов, завершая оплатой. Начинается сделка с момента, когда мы сделали шаг навстречу потребителю - например, показав ему рекламу.

Воронка действий — это способ отобразить:
- путь клиента до совершения нужного нам действия;
- долю людей, которые «не отваливаются» и переходят на каждый следующий этап этого пути.

#### Как построить воронку?

Нужно определить шаги. Например:

- Зайти на главную страницу магазина;
- Перейти на страницу товара;
- Добавить товар в корзину;
- Перейти на страницу оформления заказа;
- Оплатить заказ (целевое действие - совершение которого пользователем и есть наша цель).

![Визуальная воронка](https://pictures.s3.yandex.net/resources/voronka_1620303482.jpg)
![Пример воронки продаж](https://pictures.s3.yandex.net/resources/etap_1620303427.jpg)
![Воронка с конверсиями](https://pictures.s3.yandex.net/resources/konversiya_1620303530.png)

Вот воронка, в которой пользователь может пропускать шаги. В результате шаг, который можно пропустить, исполнили меньше пользователей, чем следующий, но обязательный.

![](https://pictures.s3.yandex.net/resources/konversiya3_1620304024.png)

Чем меньше шагов - тем лучше для пользователя, а значит - тем привлекательней для него, а значит - больше выручка.

### Воронка лидов (маркетинговая)

Фирма желает узнать, сколько будет покупателей на товар. Она открывает специальную страницу, где принимаются заявки на покупку - это "лендинг". Пользователь видит рекламу - переходит на лендинг - оставляет контакты.

Заинтересованный пользователь - это "лид". Его контакты - это тоже "лид" (второе значение). Процесс поиска заинтересованных людей ("лидов"), привлечение их (чтобы они сами интересовались) или сбор их контактов (чтобы мы им сообщили) - это "лидогенерация".

Лид можно превратить в заказчика. Когда соберётся нужное число пользователей - им предложат оформить покупку.

Цель маркетологов — собрать как можно больше лидов.

Шаги воронки:
- Показы. Сколько раз показывали баннеры (данные из рекламной системы);
- Переходы. Различают подэтапа:
  - Клики. Сколько пользователей по этим баннерам кликнуло (данные из рекламной системы);
  - Посещения (переходы). Сколько кликнувших попали на лендинг (данные из веб-аналитической системы);
- Регистрации. Сколько посетителей оставили компании свои данные (данные из веб-аналитической системы).

Данные о показах и кликах получают из рекламных систем. Информацию о переходах на лендинг и регистрациях — из системы аналитики сайта (веб-аналитики, типа Яндекс.Метрики).

Маркетинговые данные чаще всего агрегированы — отражают общее количество показов, переходов и регистраций за каждый день.

О переходе. Клик, который регистрируется в рекламной аналитике - должен автоматически привести к посещению рекламируемой страницы. Таким образом, количество кликов в рекламном отчёте должно точно совпасть с количеством переходов из этой рекламы в веб-отчёте (отчёте о посетителях страницы). Если не совпало - значит, есть техническая проблема, которую надо решить.

Системы анализа посещений веб-страниц записывают, какая страница "привела" к посещению. В случае прямого входа (по адресу, из закладки) источник посещения будет не известен. Реферал - это как раз тот, кто привёл на страницу. Обычно реферала узнают по коду в "реферальной ссылке" - это URL, к которому присобачено кодовое обозначение того, кто привёл клиента. Пользователь кликает на рекламу - де факто на реферальную ссылку - браузер запрашивает страницу по этой ссылке - сервер читает ссылку, и видит в тексте ссылки, какое объявление сработало (и у кого оно размещено).

Как только все данные собраны, можно считать конверсии.

### CTR Click-Through Rate

Клики (переходы) / показы.

### CR Convertion Rate

Регистрации / переходы (клики).

In [ ]:
# загружаем статистику рекламы
ad_data = pd.read_csv(URL + 'ad_data.csv')
ad_data.head()

In [ ]:
# и статистику сайта
site_data = pd.read_csv(URL + 'site_data.csv')
site_data.head()

In [ ]:
# объединяем данные за одни и те же даты
funnel = pd.merge(ad_data, site_data, on='date')

# рассчитываем конверсии
funnel['ctr, %'] = funnel['clicks'] / funnel['impressions'] * 100
funnel['cr, %'] = funnel ['registrations'] / funnel['clicks'] * 100

funnel.head()

#### Вычисление конверсий за несколько периодов вместе

Например, за неделю:

In [ ]:
# устанавливает подходящий тип данных для дат
funnel['date'] = pd.to_datetime(funnel['date'])
# теперь у каждой даты легко узнать номер недели в году
funnel['week'] = funnel['date'].dt.isocalendar().week

# создаём группы по неделям
funnel_weekly = funnel.groupby('week')[[
    # оставляем в каждой группе только три колонки
    'impressions', 'clicks', 'registrations'
]].sum()  # и записываем в каждой колонке сумму за всю неделю

# добавляем данные - CTR и CR для каждой недели
funnel_weekly['ctr, %'] = funnel_weekly['clicks'] / funnel_weekly['impressions'] * 100
funnel_weekly['cr, %'] = funnel_weekly['registrations'] / funnel_weekly['clicks'] * 100

funnel_weekly

И за месяц:

In [ ]:
ad_data = pd.read_csv(URL + 'ad_data_2.csv')
site_data = pd.read_csv(URL + 'site_data_2.csv')

# соединяем данные за каждую дату
funnel_daily = pd.merge(ad_data, site_data, on='date')
# добавляем CTR и CR за каждую дату
funnel_daily['ctr, %'] = funnel_daily['clicks'] / funnel_daily['impressions'] * 100
funnel_daily['cr, %'] = funnel_daily['registrations'] / funnel_daily['clicks'] * 100

# добавляем номер недели и месяца у каждой даты
funnel_daily['date'] = pd.to_datetime(funnel_daily['date'])
funnel_daily['week'] = funnel_daily['date'].dt.isocalendar().week
funnel_daily['month'] = funnel_daily['date'].dt.month

# создаём сводку по неделям
funnel_weekly = funnel_daily.groupby('week')[['impressions', 'clicks', 'registrations']].sum()
funnel_weekly['ctr, %'] = funnel_weekly['clicks'] / funnel_weekly['impressions'] * 100
funnel_weekly['cr, %'] = funnel_weekly['registrations'] / funnel_weekly['clicks'] * 100

display(funnel_weekly)

# создаём сводку по месяцам
funnel_monthly = funnel_daily.groupby('month')[['impressions', 'clicks', 'registrations']].sum()
funnel_monthly['ctr, %'] = funnel_monthly['clicks'] / funnel_monthly['impressions'] * 100
funnel_monthly['cr, %'] = funnel_monthly['registrations'] / funnel_monthly['clicks'] * 100

display(funnel_monthly)

### Воронка пользователей в сервисе ("продуктовая")

Она показывает, что делают пользователи на сайте или в приложении.

Продуктовые данные обычно «сырые»: каждая строчка в таблице — это отдельное событие. Например, «пользователь 42 открыл страницу сайта».

Например, интернет-магазин хочет увеличить выручку.

Покупатели проходят четыре этапа:
- Заходят на сайт;
- Добавляют в корзину товары;
- Оформляют заказ;
- Оплачивают покупку.

Чтобы выявить действия, которые влияют на количество продаж, построим продуктовую воронку и проанализируем, сколько пользователей доходят до каждого этапа.

Логи событий отличаются на разных сайтах, но есть три обязательных столбца:

- название события,
- дата и время события,
- идентификатор пользователя, с которым это событие произошло.

In [ ]:
events = pd.read_csv(URL + 'product_funnel_demo.csv')

events.head()

Самый простой способ построить воронку — посчитать, сколько раз наступило каждое событие. Сгруппируем датафрейм по полю event_name и посчитаем строки.

In [ ]:
events_count = events.groupby('event_name').agg({'uid': 'count'})

events_count

Событие pageview произошло 1249 раз, а событие add_to_cart — 1311 раз. Возможно, товар можно добавить, не просматривая одну страницу, а прямо с текущей.

Помни! Каждая строка - это некая сущность. Она должна быть уникальной. Например, с помощью "ключа" или ID. Вот тот предмет, чьи ID в таблице всегда _уникальны_ (не повторяются) - тот и есть главное содержимое таблицы. Если в таблице _уникальны_ ID действий (например, таймстампы) - то **это таблица действий**, а не пользователей. В ней, даже сосчитав **количество `user_id`**, ты получаешь **не количество пользователей**, а количество действий.

Как же узнать количество пользователей, которые совершали действие (один раз или много)? Считать количество _уникальных_ ID пользователей.

In [ ]:
users_count = (
    events.groupby('event_name')
    # подсчитываем именно количество пользователей (уникальных) в группе,
    .agg({'uid': 'nunique'})
    # сортируем по кол-ву пользователей
    .sort_values(by='uid', ascending=False)
)

users_count

Теперь видно, сколько человек дошли до каждого этапа продуктовой воронки.

- Примерно 74% зашедших на сайт (смотревших хоть одну страницу, pageview) пользователей добавили товар в корзину.
- Только 34% добавивших товар в корзину перешли к оплате.
- И лишь 36% начавших оформлять заказ оплатили его.

Можно попробовать это исправить: например, разослать персональные скидки пользователям, которые добавили товар в корзину, но не перешли к оформлению, а также улучшить страницу оплаты.

## Анализ по когортам

### Агрегация профилей пользователя - источника перехода, первое посещение, регистрация, первый платёж

Обычно чем больше клиентов, тем лучше для бизнеса. Задача бизнеса - принять такие решения, исполнить такие длействия, чтобы как можно больше людей принесли деньги.

Разные люди реагируют по разному. В идеале, лучше всего подстраиваться под каждого потребителя индивидуально. Там, где это ещё не достигнуто - всё равно полезно разделить людей на категории ("когорты"), чтобы каждая когорта включала людей, которые одинаково реагируют на одинаковые стимулы. Благодаря этому мы будем делать что-то для тех, кому это нравится, и не делать для тех, кому не нравится. Все довольны. Доходы выше, чем если бы мы действовали одинаково для всех.

5 шагов успешного когортного анализа:
1. Определи, какой вопрос нужно ответить. Главное в анализе - получить руководство к действию.
2. Определи метрики, отоорые могут помочь ответить на вопрос. 
3. Определи, какой признак брать, чтобюы делить на когорты. Желатльно, чтобы коготрты кардинально различались.
4. Проведи анализ. Визуализируй.
5. Убедьсь, что результаты имеют смысл. Наверное, посоветуйся со специалистом и с отвлечёнными людьми.

Заполучить клиентов можно двумя способами:
- Бесплатно. Это так называемая «органика» — пользователи, которые нашли компанию самостоятельно: по рекомендации, через поиск или случайно.
- Платно. Компания вкладывает деньги в рекламу. Рекламным сетям можно платить за каждый просмотр объявления пользователем, за клик по нему или даже за целевое действие — переход на сайт или скачивание приложения. В последнем случае бизнес фактически «покупает» новых клиентов.

Но где инвестиции, там и риски. «Купленные» клиенты могут ничего не купить. Или приобрести самый дешёвый товар и исчезнуть навсегда. Поэтому бизнес стремится привлечь самых «качественных» клиентов — тех, кто принесёт больше всего денег.

Риски платных клиентов - они могут не окупиться (не заплатить, не сделать
покупки или сделать слишком маленькую покупку).

Качественный привлечённый клиент - это такой, который:
- долго с компанией,
- рекомендует и упоминает её,
- часто покупает,
- много тратит,
- мало расходов на привлечение,
- мало расходов на удержание.

Чтобы оценить качество клиентов, применяют когортный анализ.

В основном аналитик желает увидеть:
- какие когорты охотнее делают нужное действие (например, платят), чтобы затем привлекать людей именно в эту когорту, с этими признаками;
- как изменяется намерение к нужному действию: как часто его повторяют, какие ещё тренды есть кроме целевого. Для этого изучают когорты, составленные по времени. И изучают их поведение отдельно от остальных, как и через сколько времени люди меняют поведение и отношение. Чтобы не смешивать только что пришедшего пользователя и давно освоившегося, недавно ещё бывшего на сайте, и уже давно покинувшего.

Когортный анализ редко применяют для прогнозирования поведения пользователей, ведь в будущем на него могут повлиять внешние, ещё не известные факторы.

#### Основные правила

Аналитики называют группы (категории) пользователей "когортами". Участников
когорты объединяет общий признак. Примеры: впервые посетившие страницу,
участвующие в программе лояльности, пользователи приложения.

Чем как объединяют учатников в когорту:
- входят в один временной промежуток, и связаны однотипными событиями.
- Дополнительные (персональные) признаки: возраст, пол, профессия, локация,
особенности поведения.

Например: событие - первое посещение сайта, промежуток - в апреле. Когорту
можно назвать "пользователи, впервые посетившие сайт в апреле".
А можно ввести доп признак: "пользователи из Москвы".

#### Типичный набор данных пользователя

Все системы бизнес-аналитики собирают примерно один и тот же минимальный набор информации о пользователях. Есть две большие группы данных: журнал посещений сайта пользователями, то есть пользовательских сессий, и журнал покупок.

##### Данные журнала пользовательских сессий

- `user_id` - уникальный идентификатор пользователя;
- `session_start` - дата начала сессии;
- `session_duration` или `session_end` - дата окончания или длительность сессии;
- `device` — устройство, с которого пользователь заходил на сайт;
- `region` — географическое положение в момент посещения сайта;
- `channel` —  рекламный канал, источник переходов или иной ресурс, с которого пользователь перешёл на сайт.

##### Данные журнала покупок

- уникальный идентификатор пользователя,
- дата совершения покупки,
- сумма покупки.

#### Создание профилей пользователей

1. Загрузить данные журнала посещений.
2. Для каждого пользователя определить дату и время первой сессии.
3. Для каждого пользователя определить соответствующие задаче параметры первой сессии. Например, источник перехода на сайт.

In [ ]:
# загружаем журнал сессий
sessions = pd.read_csv(URL + 'sessions.csv')

# преобразуем данные о времени для дальнейших расчётов
sessions['session_start'] = pd.to_datetime(sessions['session_start'])

sessions

In [ ]:
orders = pd.read_csv(URL + 'book_orders.csv')
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
orders

#### `get_columns_to_rename()`:

- `dataframe` — датафрейм, в котором нужно обновить имена колонок,
- `keys` - список колонок, которые нельзя переименовывать,
- `addition` - текст, который добавим к имени,
- `prefix` - дописать перед именем? если False - то после имени.

In [ ]:
def get_columns_to_rename(dataframe,
                          keys=['user_id', 'date', 'payer', 'dt'],
                          addition='first',
                          prefix=True):
    """Возвращает словарь замен старых имён колонок на новые.
    
    Служит для того, чтобы результаты обработки журнала событий
    находились в колонках с другими именами, а не с такими же,
    как хранятся в журнале событий.
    
    Это позволит объединять журналы и профили вместе,
    и не беспокоиться о том, чтобы переименовывать колонки вручную.
    Переименование проходит по интуитивно понятным правилам,
    информация будет подписана более очевидным способом,
    чем тот, что применяется в `merge()`.
    """
    source_columns = [
        x for x in dataframe.columns if not (
            x in keys or addition in x) ]
    
    # и запланируем: добавить к имени каждого столбца префикс "first_"
    # теперь наши названия содержательны и уникальны, они не смешаются
    # с названиями колонок из events, когда мы будем добавлять profiles в events
    print('Переименованные колонки:')
    columns_to_rename = dict()
    for name in source_columns:
        if prefix:
            columns_to_rename[name] = addition + '_' + name
        else:
            columns_to_rename[name] = name + '_' + addition
        print(f"'{name}' => '{columns_to_rename[name]}'")
    print()
    return columns_to_rename

#### `get_profiles()`:

- `events` — журнал событий,
- `uid` - имя колонки, которая содержит ID пользователя,
- `event_dt` - имя колонки, в которой записаны дата и время события,
- `method` - название метода (доступного столбцам pandas), которым мы найдём значение признака пользователя.

Для создания пользовательских профилей с датой первого посещения и источником перехода на сайт напишем функцию `get_profiles()`. В ней сгруппируем значения датафрейма по пользовательскому ID и применим функцию `first()`:

In [ ]:
def get_profiles(events,
                 orders=None,
                 uid='user_id',
                 event_dt='event_dt',
                 method='first'):
    """Возвращает профили пользователей на основе журнала событий.
    
    Все события группируются по пользователям. Указанным методом
    на основе всех вариантов признака выбирается нужное значение признака.
    Название метода дописывается в названия всеx преобразованныx колонок.
    """
    # получаем словарь, какие колонки в профилях как назвать
    columns = get_columns_to_rename(events, [uid], method)

    result = (
        # сортируем сессии по ID пользователя и дате посещения
        events.sort_values(by=[uid, event_dt])
        # группируем по ID
        .groupby(uid)
        # и находим характерные признаки пользователя
        .agg(method)
        # переименуем колонки, чтобы имена соответствовали содержимому
        .rename(columns=columns)
        # возвращаем uid из индекса в колонки
        .reset_index()
    )

    # определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    # эти данные понадобятся для когортного анализа
    result['date'] = result[columns[event_dt]].dt.date
    result['month'] = result[columns[event_dt]].astype('datetime64[M]')
    
    # Если мы дали ещё и журнал продаж...
    if orders is not None:
        # Это заказчик, или "пока не определился"?
        # Заказчиком считаем, если хоть однажды сделал заказ.
        # Заказ - почти всегда подразумевается "платёж".
        result['payer'] = result['user_id'].isin(orders['user_id'])

    return result

In [ ]:
pd.read_csv(URL + 'profiles.csv')

In [ ]:
# создаём профили пользователей,
# используя записи об их посещениях и покупках
profiles = get_profiles(sessions, orders, event_dt='session_start')

profiles

In [ ]:
# Отобразим, из какой страны по статистике
# лиды становятся заказчиками чаще
# то есть, где выше CR Conversion Rate
display(
    profiles.groupby('first_region')
    .agg({'payer': 'mean'})
    .sort_values('payer', ascending=False)
)

Видим, что выше всего процент плательщиков среди американцев, ниже всего - среди французов. Но разница не велика. Лучше рассмотреть больше признаков, чтобы найти таки ощутимую разницу.

Ещё, имея готовые профили пользователей, легко узнать количество привлечённых каждым источником посетителей. Достаточно сгруппировать профили по рекламному каналу и посчитать количество уникальных ID функцией `nunique()`:

In [ ]:
profiles.groupby('first_channel').agg({'user_id': 'nunique'})

In [ ]:
profiles.pivot_table(
    index='date',  # даты первых посещений
    columns='first_channel',  # источники переходов
    values='user_id',  # ID пользователей
    aggfunc='nunique'  # подсчёт уникальных значений
).plot(figsize=(15, 5), grid=True)

plt.show()

### Retention Rate

В цифровых сервисах когортный анализ применяют, чтобы сравнить «качество» пользователей. Один из главных критериев качества — как долго клиент остаётся с компанией. На языке метрик этот факт часто описывают двумя показателями: Retention Rate и Churn Rate.

Retention Rate, или коэффициент удержания, показывает, как долго клиенты остаются с компанией. Сколько пользователей из когорты относительно их изначального числа вернулись, то есть воспользовались продуктом или услугой, в последующие периоды.

Это важнейший показатель для компаний, которые зарабатывают с пользователей напрямую (интернет-магазинов или сервисов по подписке). Ведь чем дольше привлечённый клиент пользуется продуктом, тем больше потенциальная выручка.

Полезен Retention Rate и для зарабатывающих на рекламе компаний — социальных сетей, поисковых систем. Чем дольше клиенты пользуются сервисом, тем больше показов рекламных объявлений можно продать.

Чтобы узнать Retention Rate, нужно разделить количество активных пользователей когорты в нужный день на количество активных пользователей когорты на первый день.

_Например, Retention Rate на разные даты для пользователей от 1 и 2 апреля._

![Пример таблицы удержаний](https://pictures.s3.yandex.net/resources/Retention_Rate_3_1620468175.png)

Лайфтайм - это срок, который существует когорта. Единица измерения лайфтайма - длине периода, на который разделены пользователи по когортам. Если когорты образованы на основе дат - то лайфтайм - это количество полных суток с момента создания когорты. Если по неделям - то количество недель. По месяцам - количество месяцев.

Теперь Retention Rate разных когорт можно сравнивать напрямую. Или вычислить общий Retention Rate для разных когорт: складывать суммы из одинаковых лайфтаймов - а затем по полученному ряду сумм вычислять Retention Rate в тот или иной лайфтайм.

Шаги:

- Собрать таблицу событий:
  - Объединить данные сессий с профилями.
  - Рассчитать лайфтайм пользователя для каждой сессии.
- Построить таблицу долей удержания:
   - Построить таблицу удержания. То есть сводную таблицу, в которой названия строк — это даты первого посещения пользователей, названия столбцов — лайфтайм, а значения в «ячейках» — количество уникальных идентификаторов пользователей.
   - Вычислить размеры когорт и занести результаты в отдельную таблицу.
   - Разделить каждую «ячейку» таблицы удержания на соответствующий размер когорты.

In [ ]:
# в данном случае событие - это заход на сайт, сессия
# собираем данные о событиях: к данным о сессиях добавляем данные о профиле пользователя
events = sessions.merge(profiles, on='user_id', how='right')

# вычисляем лайфтайм каждого события в днях
events['lifetime'] = (
    events['session_start'] - events['first_session_start']
).dt.days

events

In [ ]:
# строим таблицу удержания - сколько пользователей из когорты повторили действие в каждый лайфтайм
retention = events.pivot_table(
    index=['date'], columns='lifetime', values='user_id', aggfunc='nunique'
)

retention.astype('Int64')

In [ ]:
# вычисляем размеры когорт
cohort_sizes = (
    # Сгруппируем данные по дате первого посещения из `dt`
    events.groupby('date')
    # и посчитаем количество уникальных пользователей в каждой когорте,
    # применив функцию `nunique` к столбцу `user_id`.
    .agg({'user_id': 'nunique'})
    # теперь это серия, которая содержит размер когорты
    .rename(columns={'user_id': 'cohort_size'})
)

cohort_sizes

In [ ]:
# делим данные таблицы удержания на размеры когорт
retention_rates = retention.div(
    cohort_sizes['cohort_size'], axis=0
)

retention_rates

#### `get_attributed_events()`

- `events` - список событий,
- `profiles` - список пользователей,
- `uid` - название колонки с ID пользователя,
- `event_dt` - название колонки, которая содержит время наступления события.

In [ ]:
def get_attributed_events(source, profile_source, uid='user_id', event_dt='event_dt'):
    """Возвращает список событий, обогащённый данными о пользователе.
    
    Эти данные полезны, чтобы собирать события в группы.
    """
    attr_events = source.merge(profile_source, on=uid, how='right')

    # вычисляем лайфтайм каждого события в днях
    attr_events['lifetime'] = (
        attr_events[event_dt] - attr_events['first_' + event_dt]
    ).dt.days
    
    return attr_events

In [ ]:
attributed_events = get_attributed_events(
    sessions,
    profiles,
    event_dt='session_start',
)

attributed_events

#### `get_rates()`

- `events` - список действий,
- `dimensions` - список признаков, по которым разделяем на когорты.

In [ ]:
def get_rates(events,
              kind='retention',
              dimensions=[]):
    """Возвращает таблицу коэффициентов.
    
    Требует список действий, обогащённый признаками пользователя,
    совершившего действия.
    И тип результата:
    - удержание (повторяющиеся действия, сумма за лайфтайм);
    - конверсия (только первые действия, накопленная сумма)
    """
    ret_kinds = ['retention']
    cr_kinds = ['conversion', 'cr']

    if (
        kind in ret_kinds
        and 'payer' in events.columns
        and not 'payer' in dimensions
    ):
        dimensions = ['payer'] + dimensions

    print('Признаки для анализа:')
    print(dimensions)
    # сколько пользователей совершили действия в каждый лайфтайм
    result = events.pivot_table(
        index=dimensions,
        columns='lifetime',
        values='user_id',
        aggfunc='nunique',
    )

    if kind in cr_kinds:
        result = result.cumsum(axis=1)

    # извлекаем размеры когорт, теперь их нет в сводке
    cohort_sizes = result.pop(0)
    
    # делим количество действовавших пользователей в каждый лайфтайм
    # на размеры когорт - а когорты сформированы в лайфтайме 0!    
    return result.div(cohort_sizes, axis=0)

In [ ]:
retention_rates = get_rates(attributed_events, dimensions=['date'])

retention_rates

### Churn Rate

Churn Rate (коэффициент "оттока") - это доля тех из когорты, кто не повторил действие в следующем периоде. Доля тех, кто перестал пользоваться услугами. Доля "ушедших" пользователей.

    Churn Rate = 1 - (current retention / previous retention)

Отличается от Retention Rate тем, что Retention Rate - это отношение результата лайфтайма к результату нулевого лайфтайма, а Churn Rate - к результату предыдущего лайфтайма.

### Учёт момента и горизонта анализа

#### Алгоритм

**Момент анализа данных** - это момент времени, который нужно отобразить в результатах анализа. Что в этот момент происходило? Как всё выглядело?

**Горизонт анализа данных** - максимальный лайфтайм, который мы включаем в анализ.

Эти ограничения вносятся, чтобы внимание аналитика сосредоточилось на вещах, которые заведомо сопоставимы. Одинаковые лайфтаймы чтобы были в каждой когорте.

Вот пример того, как умело ограничить данные, чтобы потом получить разумный результат: подсчёт среднего удержания за неделю. Естественно, берём только те когорты, которые прожили не меньше недели. Таких только две, а не все.

![Как ограничивают таблицу результатов](https://pictures.s3.yandex.net/resources/Churn_Rate_4_1620471412.png)

Чтобы учесть момент и горизонт анализа, нужно:

- Задать момент и горизонт анализа.
- Рассчитать самую позднюю подходящую дату привлечения пользователей.
- Выбрать пользователей, пришедших не позже подходящей даты.
- Выбрать события с лайфтаймом меньше (**?**) чем горизонт анализа.

**?** Вопрос: почему именно "меньше" а не "не больше"? Ведь если мы хотим знать, как поступают люди через 7 дней, то нам нужно знать, что они делают на 8-ый день. То есть, через 7 дней. То есть, включая лайфтайм, который равен горизонту.

In [ ]:
horizon = 6
observation_date = datetime.date(2019, 5, 13)

if profiles is None:
    # создаём профили пользователей, используя записи об их посещениях
    profiles = get_profiles(sessions, event_dt='session_start')

if not observation_date:
    observation_date = sessions['session_start'].dt.date.max()

# Последний приемлимый период - это дата наблюдения.
# Исходя из того, что за эту дату есть полные данные,
# это не должна быть сегодняшняя дата -
# данные за сегодня не полны, день не кончился.
# По умолчанию, последний приемлемый период - "день наблюдения".
last_suitable_acquisition_date = observation_date
# Но если задан горизонт наблюдений,
if horizon:
    # то последний приемлемый период - тот, который настал на
    # {horizon} периодов перед {observation_date}.
    last_suitable_acquisition_date -= datetime.timedelta(days=horizon - 1)

# исключаем профили, которые слишком молоды,
# чтобы прожить нужное количество периодов
# "у них максимальный лайфтайм меньше чем горизонт анализа"
suitable_profiles = profiles.query('date <= @last_suitable_acquisition_date')

# дополняем данные только теми, которые в нужных профилях
attributed_events = get_attributed_events(sessions, profile_source=suitable_profiles, event_dt='session_start')

# по умолчанию считаем, что любые лайфтаймы нас устроят
suitable_events = attributed_events
# но если указан горизонт анализа - то...
if horizon:
    # оставляем только записи с лайфтаймами,
    # которые меньше чем горизонт анализа
    suitable_events = attributed_events.query('lifetime < @horizon')

get_rates(suitable_events, dimensions=['date'])

#### `get_suitable_events()`

- `events` — данные журнала событий (сессий/посещений, регистраций, установок, покупок...),
- `dimensions` - список признаков, по которым разделяем на когорты (по умолчанию не заданы),
- `observation_date` — момент анализа (по умолчанию не задан),
- `horizon` — горизонт анализа в днях (по умолчанию не задан),
- `profiles` — профили пользователей (по умолчанию не заданы).

In [ ]:
def get_suitable_events(events,
                        dimensions=[],
                        event_dt='event_dt',
                        observation_date=None,
                        horizon=None,
                        profiles=None):
    """Возвращает список подходящих для анализа событий.
    
    В списке только события пользователей,
    проживших до горизонта анализа к моменту анализа.
    Все события дополнены признаками пользователя
    и готовы к подсчёту статистики об удержании пользователя.
    """
    if profiles is None:
        # создаём профили пользователей, используя записи об их посещениях
        profiles = get_profiles(events, event_dt=event_dt)

    if not observation_date:
        observation_date = events[event_dt].dt.date.max()

    # Последний приемлимый период - это дата наблюдения.
    # Исходя из того, что за эту дату есть полные данные,
    # это не должна быть сегодняшняя дата -
    # данные за сегодня не полны, день не кончился.
    # По умолчанию, последний приемлемый период - "день наблюдения".
    last_suitable_acquisition_date = observation_date
    # Но если задан горизонт наблюдений,
    if horizon:
        # то последний приемлемый период - тот, который настал на
        # {horizon} периодов перед {observation_date}.
        last_suitable_acquisition_date -= datetime.timedelta(days=horizon - 1)

    # исключаем профили, которые слишком молоды,
    # чтобы прожить нужное количество периодов
    # "у них максимальный лайфтайм меньше чем горизонт анализа"
    suitable_profiles = profiles.query('date <= @last_suitable_acquisition_date')

    # дополняем данные только теми, которые в нужных профилях
    attributed_events = get_attributed_events(events, profile_source=suitable_profiles, event_dt=event_dt)

    # по умолчанию считаем, что любые лайфтаймы нас устроят
    suitable_events = attributed_events
    
    # но если указан горизонт анализа - то...
    if horizon:
        # оставляем только записи с лайфтаймами,
        # которые меньше чем горизонт анализа
        suitable_events = suitable_events.query('lifetime < @horizon')

    return suitable_events

#### `get_retention()`

- `events` — данные журнала событий (сессий/посещений, регистраций, установок, покупок...),
- `dimensions` - список признаков, по которым разделяем на когорты (по умолчанию не заданы),
- `observation_date` — момент анализа (по умолчанию не задан),
- `horizon` — горизонт анализа в днях (по умолчанию не задан),
- `profiles` — профили пользователей (по умолчанию не заданы).

In [ ]:
def get_retention(events, event_dt='event_dt', dimensions=[], observation_date=None, horizon=None, profiles=None):
    """Возвращает статистику удержания пользователей.
    
    Требует только журнал событий.
    """
    suitable_events = get_suitable_events(
        events=events,
        event_dt=event_dt,
        dimensions=dimensions,
        observation_date=observation_date,
        horizon=horizon,
        profiles=profiles,
    )

    result = get_rates(suitable_events, kind='retention', dimensions=dimensions)
    return suitable_events, result

In [ ]:
events, retention = get_retention(
    events=sessions,
    event_dt='session_start',
    dimensions=['date'],
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
)

In [ ]:
retention

In [ ]:
events

### Визуализация

#### `show_heatmap_table()` - тепловая карта удержания

Хитмэп — отличный выбор, если вы хотите сравнить удержание нескольких когорт.

In [ ]:
def show_heatmap_table(retention, figsize=(15, 6)):
    """Показывает единую тепловую карту удержаний.
    """
    plt.figure(figsize=figsize)
    sns.heatmap(
        retention,
        annot=True,  # включаем подписи
        fmt='.2%',  # переводим значения в проценты
    )
    plt.title('Тепловая карта удержания')  # название графика
    plt.show()

In [ ]:
_, triangle_retention = get_retention(sessions, event_dt='session_start', dimensions=['date'])
show_heatmap_table(triangle_retention)

#### Кривая удержания

Кривые удержания подходят для «быстрого» сравнения показателей. Метод `plot()` строит график, на котором линии отражают значения каждого столбца датафрейма.

In [ ]:
retention

In [ ]:
# транспонирование (переворачивание) таблицы с помощью атрибута T:
# столбцы становятся строками, а строки - столбцами
test = retention.T

test

In [ ]:
# это способ получить названия строк
test.index.values

In [ ]:
# строим кривые удержания
retention.T.plot(
    grid=True,
    # отметки на оси X — названия колонок retention, то есть - номера лайфтаймов
    xticks=list(retention.columns),
    figsize=(15, 5),
)
plt.xlabel('Лайфтайм')  # название оси X
plt.title('Кривые удержания по дням привлечения')  # название графика
plt.show()

#### Кривые истории изменения удержаний

Если задача — проанализировать, как менялось удержание от когорты к когорте для каждого дня «жизни» пользователей, подойдёт график истории изменений.

Построить такой график проще всего — достаточно вызвать `plot()` к таблице удержания. Без всякого транспонирования.

Каждая линия на этом графике показывает, как менялось удержание пользователей на определённый лайфтайм. Например, синяя линия сверху отражает изменения в удержании второго дня или первого лайфтайма (лайфтайм 1 - прошёл один полный день, идёт второй), а нижняя фиолетовая — в удержании шестого дня.

In [ ]:
# строим графики изменений
retention.plot(grid=True, figsize=(15, 5))
plt.xlabel('Дата привлечения')
plt.title('Динамика удержания пользователей')
plt.show()

In [ ]:
profiles

In [ ]:
pd.read_csv(URL + 'profiles_backup.csv')

In [ ]:
events, retention = get_retention(
    sessions, event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['payer'],
)

In [ ]:
# строим тепловую карту
retention.T.plot(grid=True, xticks=retention.columns, figsize=(15, 5))
plt.xlabel('Лайфтайм')
plt.title('Кривые удержания двух когорт: совершавшие покупку и не совершавшие')
plt.show()

### Графики, которые отображают когорты с двумя-тремя признаками

Удержание платящих значительно выше удержания неплатящих. Так бывает почти всегда, поэтому разбивка пользователей на платящих и неплатящих — стандартная практика. Мы дополнили `get_retention`, он учитывает разбивкуй на когорты: плательщики и лиды.

In [ ]:
events, retention = get_retention(
    sessions, event_dt='session_start',
    # а следующий фрейм содержит колонку `payer`
    # если колонку или весь фрейм убрать - то разбивка
    # на тех кто платил и кто не платил - исчезнет
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['date']
)

retention

#### Тепловые карты удержания

Видим, что у нас составной индекс.

In [ ]:
show_heatmap_table(retention)

Теперь каждой строке таблицы удержания соответствуют два параметра: дата и признак совершения покупки. График и подписи заметно разрослись.

Разделим график надвое: построим по тепловой карте для каждой группы пользователей — платящих и неплатящих. Для этого вызовем функцию `figure()` из модуля `pyplot` библиотеки `matplotlib` и напишем цикл `for`.

In [ ]:
retention

In [ ]:
plt.figure(figsize=(20, 6)) # задаём размер холста для графиков

# берём порядковый номер и имя категории
# а категорий всего две: "платил" и "не платил"
for i, payer in enumerate(profiles['payer'].unique()):
    sns.heatmap(
        # из всей таблицы удержаний берём только те записи,
        # где индекс 'payer' равен значению 'payer' из итератора
        retention.query('payer == @payer')
        # удаляем индекс payer - он теперь не нужен
        .droplevel('payer'),
        # добавляем подписи значений
        annot=True,
        # переводим значения в проценты
        fmt='.2%',
        # строим каждый график в своей ячейке
        # (number_of_rows, number_of_cols, index)
        # index starts at 1 in the upper left corner and increases to the right.
        # index can also be a two-tuple specifying the (first, last) indices
        # (1-based, and including last) of the subplot,
        # e.g., fig.add_subplot(3, 1, (1, 2)) makes a subplot
        # that spans the upper 2/3 of the figure.
        ax=plt.subplot(1, 2, i + 1),
    )
    # задаём названия графиков с учётом значения payer
    plt.title('Тепловая карта удержания для payer = {}'.format(payer))

plt.tight_layout()  # «подгоняем» размер графиков, чтобы уместились подписи
plt.show()

В программе для построения двух хитмэпов мы передали функции `subplot()` аргументы `1`, `2` и `i + 1`: в таблице графиков одна строка и два столбца. 

```python
        ax=plt.subplot(1, 2, i + 1)
```

Переменная `i` принимает значения `0` и `1`, а нумерация ячеек в таблице графиков начинается с единицы, поэтому значение `i` увеличиваем на один. Так первый график окажется в первой ячейке, а второй — во второй.

#### Кривые удержания

Проанализируем удержание с разбивкой когорт не по дате, а по другому параметру — устройству, с которого пользователи впервые зашли на сайт. Эта информация сохранена в столбце `device` — добавим его в параметр `dimensions`. Горизонт и момент анализа данных остаются прежними. Вызовем функцию `get_retention()` и построим хитмэп.

In [ ]:
_, retention = get_retention(
    sessions,
    event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['device'],
)

retention

In [ ]:
plt.figure(figsize=(15, 6))
sns.heatmap(retention, annot=True, fmt='.2%')
plt.title('Тепловая карта удержания')
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))
for i, payer in enumerate(profiles['payer'].unique()):
    retention.query('payer == @payer').droplevel('payer').T.plot(
        grid=True,
        xticks=retention.columns,
        ax=plt.subplot(1, 2, i + 1),
    )
    plt.xlabel('Лайфтайм')
    plt.title('Кривые удержания для payer = {}'.format(payer))

plt.show()

По правому графику с кривыми удержания неплатящих пользователей сложно определить, пользователи каких устройств удерживаются хоть сколько-то лучше.

#### Кривые истории изменений удержания

Избежать слияния линий позволит третий изученный вами способ визуализации — график истории изменений. На таком графике каждая линия соответствует определённому лайфтайму, а по горизонтальной оси отмечены даты привлечения пользователей. 

Однако построить его теперь непросто: после добавления параметра `dimensions` даты привлечения пользователей пропали из таблицы удержания.

История изменений удержания - это по сути анализ когорт, у которых главный признак - период, в который когорта пришла. Даты становятся обязательным признаком.

In [ ]:
_, retention = get_retention(
    sessions, event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['device', 'date'],
)

retention

Получаем таблицу удержания, сгруппированную по трём признакам:

- Совершение покупок. Столбец `payer`, значение `True` или `False`.
- Устройство, с которого пользователь просматривает сайт. Столбец `device`, значения `Android`, `Mac`, `PC` или `iPhone`.
- Дата привлечения пользователей. Столбец `dt` с датами от 1 до 8 мая 2019 года.

In [ ]:
# холст для графиков будет такого размера
plt.figure(figsize=(20, 8))

# на холст графики станут в столько рядов,
# сколько вариантов в 'payer'
nrows = len(profiles['payer'].unique())
# и столько столбцов, сколько вариантов в 'payer'
ncols = len(profiles['first_device'].unique())

# наружний цикл - перебор строк, рядов
for i, payer in enumerate(profiles['payer'].unique()):
    # внутренний цикл - перебо столбцов, позиций в ряду
    for j, device in enumerate(profiles['first_device'].unique()):
        (
            # оставляем записи, в которых нужный статус плательщика и нужное устройство
            retention.query('payer == @payer and device == @device')
            .droplevel(['payer', 'device'])
            .plot(
                grid=True,
                # количество рядов и ячеек в ряду - берутся из констант, заданных до цикла
                # номер ячейки, куда запишем диаграмму - вычисляется
                # на основе порядковых номеров итераций циклов
                ax=plt.subplot(nrows, ncols, i * ncols + j + 1),
                rot=30,
            )
        )
        plt.xlabel('Дата привлечения')
        plt.title('Удержание для payer = {} на {}'.format(payer, device))

# Adjust the padding between and around subplots
plt.tight_layout()
plt.show()

#### Неудачные кривые удержания - когда признаков больше двух

В принципе, это же можно увидеть и на обычных линейных графиках удержания. Построим их, запустив код, который мы использовали для одного дополнительного признака, но теперь  - с двумя дополнительными.

In [ ]:
plt.figure(figsize=(20, 6))  # задаём размер сетки

for i, payer in enumerate(profiles['payer'].unique()):
    retention.query('payer == @payer').droplevel('payer').T.plot(
        grid=True,
        xticks=retention.columns,
        ax=plt.subplot(1, 2, i + 1), # задаём расположение графиков
    )
    plt.xlabel('Лайфтайм')
    plt.title('Кривые удержания для payer = {}'.format(payer))

plt.show()

Итак, каждая линия - когорта по трём признакам. Один признак - платежи, по ним когорты распределены вправо и влево. В целом, было бы удобнее распределить их так, как в примере с историей удержания - по горизонтали устройста, по вертикали - наличие платежей.

#### Удачные графики удержания, если много двух признаков

Чтобы построить кривые удержания с разбивкой по совершению покупок и устройствам, параметру `dimensions` при вызове функции `get_retention()` нужно передать только столбец `device`. Перегруппировать текущую таблицу уже не выйдет: для группировки нужны сырые данные, а не готовые коэффициенты.

На практике аналитик почти всегда хочет видеть и кривые удержания, и графики истории изменений. Чтобы иметь доступ и к тому, и к другому виду графиков, можно вызвать `get_retention()` дважды — с разным набором столбцов в параметре `dimensions`.


In [ ]:
# один вызов для построения кривых удержания
events, retention = get_retention(
    sessions, event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['device'], 
)

# и другой — для построения графиков динамики удержания
events, retention_history = get_retention(
    sessions, event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['device', 'date'], 
)

Однако это не лучший вариант. 

Во-первых, мы дважды создаём один и тот же датафрейм с сырыми данными `events`. Это замедляет работу программы. 

Во-вторых, за одно исследование аналитик обычно рассчитывает удержание не один, а множество раз — с разбивкой когорт по совершенно разным признакам. Каждый раз вызывать одну и ту же функцию дважды неудобно.

Гораздо эффективнее за один вызов `get_retention()` получать сразу три таблицы:

- события с атрибутами,
- удержание,
- динамика удержания.

Таблица динамики удержания отличается от простой таблицы удержания обязательной группировкой по дополнительному признаку — дате привлечения пользователей. А вот таблица удержания может не содержать этого признака. Добавим в тело функции `get_retention()` создание таблицы `retention_in_time`, которая будет сгруппирована по всем признакам из `dimensions`, а также столбцу `date`.

#### `get_retention_hist()` (using `date` in data)

In [ ]:
def get_retention_hist(events, event_dt='event_dt',
                       dimensions=[],
                       observation_date=None,
                       horizon=None,
                       profiles=None):
    """Возвращает статистику удержания, дополняя разбивкой по дням.
    
    Это можно использовать, чтобы дополнить графики
    графиками истории удержания.
    """
    # получаем подготовленные данные о событиях и основную статистику
    suitable_events, retention = get_retention(
        events=events,
        event_dt=event_dt,
        dimensions=dimensions,
        observation_date=observation_date,
        horizon=horizon,
        profiles=profiles,
    )
    
    # готовим и историю удержания
    # по умолчанию история удержания пуста
    retention_in_time = None
    # но если есть поле с датами, то...
    if 'date' in suitable_events:
        # история удержания - это таблица удержания с ещё одним признаком - датой
        retention_in_time = get_rates(
            suitable_events,
            dimensions=(dimensions + ['date'])
        )

    return suitable_events, retention, retention_in_time

#### Автопостроение холстов с графиками

##### `show_retention_line_plot()`

In [ ]:
def show_retention_line_plot(retention, figsize=(20, 8)):
    """Показывает графики удержания. 
    """
    plt.figure(figsize=figsize)
    
    # retention.droplevel(-1).index.names:
    # retention.index.names[0]
    
    # указываем, какой признак "распределим" по горизонтальным графикам
    dimension = 'payer'
    # и какие у него уникальные варианты
    unique_values = retention.index.get_level_values(dimension).unique()
    # зафиксируем, сколько графиков будет в одном ряду
    ncols = len(unique_values)

    for i, value in enumerate(unique_values):
        (
            # оставляем для графика только те данные,
            # которые соответствуют нужному значению
            retention.query('payer == @value')
            # выкидываем не нужный в этой пикче уровень
            .droplevel(dimension)
            # транспонируем и рисуем график
            .T.plot(
                grid=True,
                xticks=list(retention.columns),
                ax=plt.subplot(1, ncols, i + 1),
            )
        )
        plt.xlabel('Лайфтайм')
        plt.title('Кривые удержания для {} = {}'.format(dimension, value))
    plt.show()

##### `get_retention_hist_plot()`

In [ ]:
def show_retention_hist_plot(retention_history, figsize=(20, 8)):
    """Показывает графики истории удержания.
    """
    plt.figure(figsize=figsize)
    
    row_dimension = 'payer'
    col_dimension = 'device'
    
    unique_row_values = retention_history.index.get_level_values(row_dimension).unique()
    unique_col_values = retention_history.index.get_level_values(col_dimension).unique()

    nrows = len(unique_row_values)
    ncols = len(unique_col_values)

    for i, row_value in enumerate(unique_row_values):
        for j, col_value in enumerate(unique_col_values):
            (
                retention_history.query('payer == @row_value and device == @col_value')
                .droplevel([row_dimension, col_dimension])
                .plot(
                    grid=True,
                    rot=30,
                    ax=plt.subplot(nrows, ncols, i * ncols + j + 1),
                )
            )
            plt.xlabel('Дата привлечения')
            plt.title('Удержание для {} = {} на {}'.format(row_dimension, row_value, col_value))
    plt.tight_layout()
    plt.show()

##### `show_retention()`

In [ ]:
def show_retention(*args, **kwargs):
    """Даёт полный обзор удержания.
    """
    events, retention, retention_history = get_retention_hist(*args, **kwargs)
    display(events)
    display(retention)
    display(retention_history)
    show_heatmap_table(retention)
    show_retention_line_plot(retention)
    show_retention_hist_plot(retention_history)

In [ ]:
show_retention(
    sessions, event_dt='session_start',
    profiles=profiles,
    observation_date=datetime.date(2019, 5, 13),
    horizon=6,
    dimensions=['device'],
)

#### Conversion Rate в когортном анализе

Итак, конверсия - это доля людей, перешедших в новый этап, из состояния в состояние. Коэффициент конверсии когорты - это доля когорты, совершившей действие.

Чаще всего речь об оплате: из "неплатящих" в платящие. Когда растёт доля платящих - обычно это хорошо.

![Число плательщиков в выбранной когорте пользователей](https://pictures.s3.yandex.net/resources/Conversion_Rate_2_1620486051.png)

Смотрим внимательно: в таблице - число _первых_ покупок в лайфтайм в когорте. Ниже - общее число число _первых_ покупок в этой когорте, с начала наблюдений до завершения текущего лайфтайма.

Convertion Rate = накопленное число "конвертировавшихся" из когорты / размер когорты.

![Конверсия в когорте](https://pictures.s3.yandex.net/resources/Conversion_Rate_4_1620486122.png)

Тут тоже важны _горизонт наблюдения_ и _дата наблюдения_.

![Треугольная таблица](https://pictures.s3.yandex.net/resources/Conversion_Rate_5_1620486401.png)

![Её обработка](https://pictures.s3.yandex.net/resources/Conversion_Rate_6_1620486529.png)

Расчёт конверсии при когортном анализе очень похож на расчёт коэффициентов удержания и оттока. Отличие — в _исходных данных_. Если для удержания и оттока важно _количество активных пользователей_ в каждый из дней «жизни», то для конверсии — _количество первых покупок_.

Как рассчитать Conversion Rate по когортам:
1. Получить пользовательские профили и данные о покупках.
2. Найти дату и время первой покупки для каждого пользователя.
3. Добавить данные о покупках в профили.
4. Рассчитать лайфтайм пользователя для каждой покупки.
5. Построить таблицу конверсии - сводную таблицу, в которой:
  - названия строк — это даты первого посещения пользователей,
  - названия столбцов — лайфтайм,
  - а значения в «ячейках» — количество уникальных идентификаторов пользователей.
6. Посчитать сумму с накоплением для каждой строки таблицы конверсии.
7. Вычислить размеры когорт и занести результаты в отдельную таблицу.
8. Разделить каждую «ячейку» таблицы конверсии на соответствующий размер когорты.

Если взглянуть, то понятно, что данные для конверсии (платежи) устроены так же, как данные для удержания (посещения). И профили достраиваются так же - поиском первых по времени событий. И прибавляются к событиям профили так же. Итого, шаги 1...4 делаются так: к платежам применить тот же `get_suitable_events()`.

5 и 6 шаги: способ группировки и aggfunc() при вычислениях конверсии - другой. Его и напишем. Может, потом оформить `get_retention()` и `get_conversion()` как единый декоратор над `get_retention/convertion_rates()`.

7 и 8 шаги: по сути как в удержании.

In [ ]:
sessions.info()

In [ ]:
orders.info()

In [ ]:
profiles.info()

In [ ]:
# выбираем покупки
suitable_purchases = get_suitable_events(
    orders,
    # добавляем их в профили, созданные на основе сессий
    profiles=get_profiles(
        # делаем так, чтобы время события у сессий и покупок
        # было под одинаковыми названиями - так их проще сравнить
        sessions.rename(columns={'session_start': 'event_dt'})
    )
)

suitable_purchases.head()

In [ ]:
get_rates(suitable_purchases, kind='conversion', dimensions=['first_region'])

In [ ]:
dimensions = ['first_region']

# сколько пользователей совершили действия в каждый лайфтайм
result = suitable_purchases.pivot_table(
    index=dimensions,
    columns='lifetime',
    values='user_id',
    aggfunc='nunique',
)

result

In [ ]:
result = result.cumsum(axis=1)

result

In [ ]:
cohort_sizes = suitable_purchases.groupby('first_region').agg({'event_dt': 'count'})
cohort_sizes

In [ ]:
result.div(cohort_sizes['event_dt'], axis=0)

In [ ]:
def get_conversion(
    profiles,
    purchases,  # заменили sessions
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # Шаг 1. Получить пользовательские профили и данные о покупках
    # передаём их в качестве аргументов profiles и purchases

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - datetime.timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('date <= @last_suitable_acquisition_date')

    # Шаг 2. Найти дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # Шаг 3. Добавить данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # Шаг 4. Рассчитать лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_session_start']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):

        # Шаг 5. Построить таблицу конверсии
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )

        # Шаг 6. Посчитать сумму с накоплением для каждой строки
        result = result.fillna(0).cumsum(axis = 1)

        # Шаг 7. Вычислить размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )

        # Шаг 8. Объединить таблицы размеров когорт и конверсии
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)

        # Шаг 9. Разделить каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)

        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['date'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
profiles.head(5)

In [ ]:
orders.head(5)

В столбце `event_dt` датафрейма `orders` хранятся дата и время совершения покупки — как и предусмотрено в коде функции `get_conversion()`.

Рассчитаем конверсию с разбивкой по регионам, передав `get_conversion()` фреймы `profiles` и `orders`, а также столбец `region` в качестве параметра `dimensions`, и построим тепловую карту по таблице конверсии. Момент и горизонт анализа данных остаются прежними — 13 мая 2019 года и 6 дней соответственно.

In [ ]:
conversion_raw, conversion, conversion_history = get_conversion(
    profiles, orders, datetime.date(2019, 5, 13), 6, dimensions=['first_region']
)

In [ ]:
plt.figure(figsize=(15, 5))
sns.heatmap(conversion.drop(columns=['cohort_size']), annot=True, fmt='.2%')
plt.title('Тепловая карта конверсии по странам')
plt.show() 

In [ ]:
plt.figure(figsize = (20, 5)) # задаём размер «подложки»

# исключаем размеры когорт
# конверсии первого дня различаются, их удалять не нужно
report = conversion.drop(columns = ['cohort_size'])

sns.heatmap(
    report, annot=True, fmt='.2%', ax=plt.subplot(1, 2, 1)
)  # в первой ячейке таблицы графиков строим тепловую карту
plt.title('Тепловая карта конверсии по странам')

report.T.plot(
    grid=True, xticks=list(report.columns.values), ax=plt.subplot(1, 2, 2)
)  # во второй — кривые конверсии
plt.title('Кривые конверсии по странам')

plt.show()

In [ ]:
# считаем конверсию без параметра dimensions
conversion_raw, conversion, conversion_history = get_conversion(
    profiles, orders, datetime.date(2019, 5, 13), 6
)

# строим хитмэп по таблице конверсии
sns.heatmap(conversion.drop(columns=['cohort_size']), annot=True, fmt='.2%')
plt.title('Тепловая карта конверсии без разбивки')
plt.show()

In [ ]:
events, conversion, conversion_hist = get_conversion(
    profiles, orders, datetime.date(2019, 5, 13), 6, dimensions=['first_region']
)
conversion_hist

In [ ]:
plt.figure(figsize=(20, 5))

report = conversion.drop(columns=['cohort_size'])
report.T.plot(
    grid=True, xticks=list(report.columns), ax=plt.subplot(1, 2, 1)
)
plt.title('Конверсия первых шести дней с разбивкой по странам')

# для графика истории изменений преобразуем таблицу динамики конверсии
report = (
    conversion_hist[1]
    .reset_index()
    .pivot_table(index='date', columns='first_region', values=1, aggfunc='mean')
    .fillna(0)
)

report.plot(
    grid=True, ax=plt.subplot(1, 2, 2)
)
plt.title('Динамика конверсии второго дня с разбивкой по странам')

plt.show()

## Юнит-экономика

Сколько зарабатывает бизнес с одного объекта.

Описывает, как:
- рассчитывать экономику одной продажи;
- определять, при каком объёме продаж бизнес выйдет в плюс;
- выбирать подходящую модель оплаты рекламы;
- считать пожизненную ценность, стоимость привлечения и окупаемость клиентов;
- быстро визуализировать основные метрики.

### С покупателя

Инвестиции в маркетинг строятся по принципу: «Вкладываем деньги в рекламу и получаем новые заказы. Если прибыль с заказа выше, чем затраты на его получение, значит, всё хорошо». Но на самом деле компании привлекают не заказы — привлекают покупателей, которые заказывают.

Главный принцип успешных инвестиций - полученный доход должен превысить затраты.

Чтобы включить в анализ повторные покупки, считают экономику одного покупателя. Три самые важные метрики в этом методе — LTV, CAC и ROI.

Для всех расчётов в когорте по-прежнему действуют правила. По-прежнему нужно выбирать когорты не моложе чем нужно, и выбирать лайфтаймы не больше чем нужно.

### LTV Lifetime Value (ARPU Average Revenue Per User)

Чем больше клиент дал денег - тем он лучше для нас.

Это общая сумма денег, которую один клиент в среднем приносит компании со всех своих покупок. В теории эта метрика включает все прошлые, нынешние и будущие покупки пользователя. На практике чаще анализируют LTV за определённый срок — первые 1, 3, 7 и 14 дней после регистрации. Другое название - ARPU - Average revenue per user.

    LTV = накопленная выручка с людей когорты / объём когорты

_Выручка от пользователей в каждый из лайфтаймов._
![Выручка](https://pictures.s3.yandex.net/resources/LTV_2_1643809761.png)

_Сводка по накопленной выручке в каждой когорте._
![Сводка](https://pictures.s3.yandex.net/resources/LTV_3_1620494437.png)

_Сводка по LTV в каждой когорте._
![Сводка LTV](https://pictures.s3.yandex.net/resources/LTV_4_1620494466.png)


#### ARPPU Average Revenue Per Paying User

    ARPPU = Выручка с когорты / количество тех из когорты, кто сделал хоть одну покупку.

Иногда используются вместо LTV. За счёт исключения неплатящих пользователей ARPPU часто более показателен, чем ARPU, но считать его труднее: во-первых, число платящих в когорте со временем растёт, а во-вторых, для расчёта требуются данные о том, делал ли покупку конкретный пользователь.

Выбор между ARPU и ARPPU **_отчасти__ зависит от способа оплаты рекламы**, которым пользуется компания. Самые распространённые — СPM, СPC, CPL/CPI и CPA.

**ARPU:**
- **CPM cost per mille** - сложно учитывать, потому что не каждый просмотр вызывает переход.
- **CPC cost per click** - оплаченный пользователь зашёл на сайт, ему можно закрепить куки, которые потом позволят понять, кто он, из какой когорты. Правда, куки не точны, теряются и блокируются/уничтожаются.
**ARPPU:**
- **CPL cost per lead, CPI cost per install** - оплата за пользователя, оставившего контакты или установившего прилогу. Идентификатор - логин / почта / телефон / рекламный ID устройства. Предпочитают её, а не CPA.
- **CPA cost per action** - оплата за действие (покупку, подписку, голос или ещё что то). Внедрять сложно, но подсчёт эффективности самый ясный.


#### Алгоритм
- Получить пользовательские профили и данные о покупках.
- Добавить данные о покупках в профили.
- Рассчитать лайфтайм пользователя для каждой покупки.
- Построить таблицу выручки. То есть сводную таблицу, в которой названия строк — это даты первого посещения пользователей, названия столбцов — лайфтайм, а значения в «ячейках» — выручка.
- Посчитать сумму с накоплением для каждой строки таблицы выручки.
- Вычислить размеры когорт и занести результаты в отдельную таблицу.
- Объединить таблицы размеров когорт и выручки.
- Посчитать LTV: разделить каждую «ячейку» таблицы выручки на соответствующий размер когорты.



In [ ]:
profiles = profiles.rename(columns={'first_session_start': 'first_ts'})
profiles

In [ ]:
orders

In [ ]:
def get_ltv(
    profiles,  # Шаг 1. Получить профили и данные о покупках
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - datetime.timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('date <= @last_suitable_acquisition_date')

    # Шаг 2. Добавить данные о покупках в профили

    result_raw = result_raw.merge(
        # добавляем в профили время совершения покупок и выручку
        purchases[['user_id', 'event_dt', 'revenue']],
        on='user_id',
        how='left',
    )

    # Шаг 3. Рассчитать лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):

        # Шаг 3. Построить таблицу выручки
        # строим «треугольную» таблицу
        result = df.pivot_table(
            index=dims,
            columns='lifetime',
            values='revenue',  # в ячейках — выручка за каждый лайфтайм
            aggfunc='sum',
        )

        # Шаг 4. Посчитать сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)

        # Шаг 5. Вычислить размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )

        # Шаг 6. Объединить размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)

        # Шаг 7. Посчитать LTV
        # делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу LTV
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики LTV убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []
    # получаем таблицу динамики LTV
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['date'], horizon_days
    )

    # возвращаем обе таблицы LTV и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
ltv_raw, ltv, ltv_history = get_ltv(
    profiles, orders, datetime.date(2019, 5, 13), 7
)

sns.heatmap(ltv.drop(columns=['cohort_size']), annot=True, fmt='.2f')
plt.title('Тепловая карта LTV без разбивки')
plt.xlabel('Лайфтайм')
plt.show()

LTV за неделю после привлечения составил 0,81 доллара на пользователя.

In [ ]:
# строим кривые LTV

ltv_raw, ltv, ltv_history = get_ltv(
    profiles, orders, datetime.date(2019, 5, 13), 7, dimensions=['first_region']
)

report = ltv.drop(columns=['cohort_size'])
report.T.plot(grid=True, figsize=(10, 5), xticks=list(report.columns))
plt.title('LTV с разбивкой по странам')
plt.ylabel('LTV, $')
plt.xlabel('Лайфтайм')
plt.show()

In [ ]:
# график истории изменений LTV

ltv_raw, ltv, ltv_history = get_ltv(
    profiles, orders, datetime.date(2019, 5, 13), 7
)

report = ltv_history[[0, 4, 6]]
report.plot(grid=True, figsize=(10, 5))

plt.title('Динамика LTV 1-го, 5-го и 7-го дней жизни')
plt.ylabel('LTV, $')
plt.xlabel('Даты привлечения пользователей')
plt.show()

In [ ]:
events, retention, retention_hist = get_retention_hist(
    sessions,
    profiles=profiles.rename(columns={'first_ts': 'first_session_start'}),
    observation_date=datetime.date(2019, 5, 13),
    horizon=7,
    event_dt='session_start',
)

report = retention_hist.query('payer == True').droplevel('payer')[[4, 6]]
report.plot(grid=True, figsize=(10, 5))
plt.title('Динамика удержания 5-го и 7-го дней жизни')
plt.ylabel('Удержание')
plt.xlabel('Дата привлечения пользователей')
plt.show()

Получается, в когортах с 3 по 5 мая - удержание маленькое. LTV нулевого дня в тех когортах велико, а вот пятого и седьмого дня - маленькие.

### CAC Customer Acquisition Cost

Это стоимость привлечения одного клиента. Сумма денег, в которую компании обходится каждый новый клиент.

CAC = Расходы на рекламу (инвестиции в маркетинг), привлёкшую когорту / размер когорты

Считается, что реклама (холодная) нужна только для формирования когорты, не для её удержания. Поэтому принято, что CAC окончательно рассчитывается в момент, как только когорта сформирована (закончился лайфтайм 0), и далее CAC этой когорты не меняется. Это **константа** для когорты.

_Сводка: LTV и дописанные перед ними расходы на рекламу и CAC._
![](https://pictures.s3.yandex.net/resources/LTV_5_1620494721.png)
_Вообще, поскольку есть полные расходы - логично будет дописать полные доходы с когорты._

Можно сравнивать CAC и LTV в лоб.

_График LTV по сравнению с CAC._
![](https://pictures.s3.yandex.net/resources/graph_2_1620494745.png)
_Когорта 1 апреля окупилась к концу второго дня существования. Когорта 2 апреля не окупилась по итогам семи дней. Судя по графику, маркетологи дорого купили трафик (в маркетинге так называют поток посетителей)._

#### Алгоритм

- Передать функции для создания профилей данные о тратах на рекламу.
- Объединить данные о тратах на рекламу и новых пользователях.
- Вычислить CAC: разделить рекламные расходы на количество новых пользователей.
- Добавить CAC для каждой даты привлечения и источника в профили.

In [ ]:
costs = pd.read_csv(URL + 'ad_costs_new.csv')
costs['date'] = pd.to_datetime(costs.pop('dt')).dt.date
costs.head()

In [ ]:
profiles = profiles.rename(columns={'first_channel': 'channel'})
profiles

In [ ]:
costs.merge(
    profiles.groupby(['date', 'channel']).agg({'user_id': 'nunique'}),
    on=['date', 'channel'],
)

In [ ]:
# добавляем параметр ad_costs — траты на рекламу
def get_cac_profiles(sessions, orders, events, ad_costs, event_names=[]):

    # сортируем сессии по ID пользователя и дате привлечения
    # группируем по ID и находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
         # время первого посещения назовём first_ts
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()  # возвращаем user_id из индекса
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['date'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # добавляем флаги для всех событий из event_names
    for event in event_names:
        if event in events['event_name'].unique():
            # проверяем, встречается ли каждый пользователь
            # среди тех, кто совершил событие event
            profiles[event] = profiles['user_id'].isin(
                events.query('event_name == @event')['user_id'].unique()
            )

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['date', 'channel'])
        .agg({'user_id': 'nunique'})
         # столбец с числом пользователей назовём unique_users
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()  # возвращаем dt и channel из индексов
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    # по дате и каналу привлечения
    ad_costs = ad_costs.merge(new_users, on=['date', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    # результаты сохраним в столбец acquisition_cost (CAC)
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавим стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['date', 'channel', 'acquisition_cost']],
        on=['date', 'channel'],
        how='left',
    )

    # органические пользователи не связаны с данными о рекламе,
    # поэтому в столбце acquisition_cost у них значения NaN
    # заменим их на ноль, ведь стоимость привлечения равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)
    
    return profiles  # возвращаем профили с CAC

In [ ]:
events = pd.read_csv(URL + 'events.csv')
events

In [ ]:
profiles = get_cac_profiles(sessions, orders, events, costs)
profiles.head()

Выясним, как меняется стоимость привлечения для каждого источника от когорты к когорте.

Для этого построим сводную таблицу, в которой:
- названиями строк будут даты привлечения пользователей,
- названиями столбцов — каналы привлечения,
- а значениями — средний CAC,

и построим по ней график истории изменений.

In [ ]:
# строим график истории CAC по каналам привлечения

profiles.pivot_table(
    index='date', columns='channel', values='acquisition_cost', aggfunc='mean'
).plot(figsize=(10, 5), grid=True)
plt.title('Динамика CAC по каналам привлечения')
plt.xlabel('Дата привлечения')
plt.ylabel('CAC, $')
plt.show()

### ROI Return Of Investments

Когда сравниваешь много когорт по LTV и CAC одновременно - то линий вдвое больше чем надо. Ты по сути хочешь сравнить эффективность, окупаемость. Главный принцип успешных инвестиций — затраты не должны превышать полученный в результате доход.

ROI, или Return On Investment, — окупаемость инвестиций. В экономике одного покупателя эта метрика показывает, на сколько процентов LTV превысил CAC. Ещё говорят: на сколько процентов «окупились» клиенты.

ROI = LTV / CAC = выручка с когорты / расходы на привлечение когорты

Почему не включаем в формулу себестоимость товаров? Потому что её учёт и подсчёт намного сложнее, требует неоправданно много усилий и времени.

_CAC и ROI (записанный на месте LTV)_
![](https://pictures.s3.yandex.net/resources/ROI_1_1620494905.png)



#### Алгоритм

- выбрать признак
- сосчитать LTV по признаку
- сосчитать CAC по признаку
- разделить таблицу LTV на серию CAC

In [ ]:
costs.head()

In [ ]:
orders.head()

In [ ]:
ltv_raw, ltv, ltv_hist = get_ltv(
    profiles, orders, datetime.date(2019, 5, 13), 7, dimensions=['channel']
)

display(ltv)

In [ ]:
# кривые LTV
report = ltv.drop(columns=['cohort_size'])
report.T.plot(grid=True, figsize=(10, 5), xticks=list(report.columns))
plt.title('LTV с разбивкой по источникам')
plt.xlabel('Лайфтайм')
plt.ylabel('LTV, $')
plt.show()

In [ ]:
ltv_raw.head()

In [ ]:
# самая поздняя подходящая дата привлечения из событий LTV
max_acquisition_dt = ltv_raw['date'].max()
# выбиваем только профили, которые старше этой даты
ltv_profiles = profiles.query('date <= @max_acquisition_dt')
# сколько пользователей в каждый лайфтайм?
ltv_profiles.groupby('date').agg({'user_id': 'nunique'})

In [ ]:
cac = (
    ltv_profiles.groupby('channel')
    .agg({'acquisition_cost': 'mean'})
    .rename(columns={'acquisition_cost': 'cac'})
)

cac

In [ ]:
roi = ltv.div(cac['cac'], axis=0)
roi

Столбец c размерами когорт «сломался», а ROI органических пользователей устремился в бесконечность — из-за деления на ноль.

Затраты на привлечение органических пользователей нулевые, поэтому они всегда окупаются. А раз так, исключим их — удалим из результата все строки, в которых размер когорты равен бесконечности, применяя метод `isin()` и оператор `~` ("not"). Сравнивать значения с бесконечностью в Python позволяет переменная `inf` из библиотеки `numpy`.

In [ ]:
roi = roi[~roi['cohort_size'].isin([np.inf])]
roi

In [ ]:
roi.loc[:, 'cohort_size'] = ltv.loc[:, 'cohort_size']
roi

Таблица ROI готова.

Построим кривые ROI и добавим на график уровень окупаемости, вызвав функцию [`axhline()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axhline.html) из библиотеки `matplotlib`:

- `y` — координата линии по вертикальной оси,
- `color` — цвет линии,
- `linestyle` — стиль линии,
- `label` — подпись.

Уровень окупаемости установим на уровне `1`, линию сделаем красной (`color='red'`) и пунктирной (`linestyle='--'`). Чтобы добавить её в легенду, вызовем метод [`legend()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html) библиотеки `matplotlib`.

In [ ]:
report = roi.drop(columns=['cohort_size'])
report.T.plot(grid=True, figsize=(10, 5), xticks=list(report.columns.values))

plt.title('ROI с разбивкой по каналам привлечения')
plt.ylabel('ROI')
plt.xlabel('Лайфтайм')
plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
plt.legend()
plt.show()

Судя по графику, реклама в «Яндексе» окупилась в среднем на 200%, а вот реклама в `AnotherSource` не окупилась вовсе.

### Функция для расчёта LTV, CAC и ROI

Чтобы всякий раз не повторять действия выше при расчёте ROI по новым данным, добавим их в функцию для расчёта пожизненной ценности `get_ltv()`:

- рассчитаем CAC,
- разделим LTV на CAC,
- удалим строки с бесконечным ROI.

In [ ]:
def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - datetime.timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('date <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # сохраняем в датафрейм данные пользователей и значения CAC, 
        # добавив параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['date'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

In [ ]:
# рассчитываем LTV и ROI

ltv_raw, ltv, ltv_history, roi, roi_history = get_ltv(
    profiles, orders, datetime.date(2019, 5, 13), 7, dimensions=['channel']
)

roi  # таблица ROI

In [ ]:
roi_history

In [ ]:
roi_history.pivot_table(
    index='date', columns='channel', values='cac', aggfunc='mean'
).plot(grid=True, figsize=(10, 5))
plt.title('Динамика CAC по источникам')
plt.xlabel('Даты привлечения')
plt.ylabel('CAC, $')
plt.show()

Кроме того, таблица динамики ROI позволяет оценить изменения окупаемости в зависимости от канала и даты привлечения.

Построим график динамики ROI первого дня с разбивкой по каналам.

Для этого создадим сводную таблицу, в которой
- названиями строк окажутся даты привлечения пользователей,
- названиями столбцов — каналы,
- а значениями — среднее значение ROI по столбцу 0, в котором лежат данные за нулевой лайфтайм, или первый день «жизни».

In [ ]:
roi_history.pivot_table(
    index='date', columns='channel', values=0, aggfunc='mean'
).plot(grid=True, figsize=(10, 5))
plt.title('Динамика окупаемости в первый день когорт')
plt.xlabel('Даты привлечения')
plt.ylabel('ROI')
plt.axhline(label='Уровень окупаемости', y=1, color='red', linestyle='--')
plt.show()

График показывает, что реклама в Яндексе всегда окупалась в первый же день, реклама в ином источнике - никогда.

### Как проверить правдоподобость результата

Причины неверных результатов:
- "сломанные" данные;
- неучтённые момент и горизонт анализа;
- ошибки алгоритма;
- и другое.

#### Удержание Retention Rate:

- Сумма размеров когорт равна числу новых клиентов в изучаемый период.
- Сумма размеров платящих когорт равна числу покупателей в изучаемый период.
- Удержание убывает по [экспоненте](https://ru.wikipedia.org/wiki/Экспонента).
- Удержание неплатящих убывает быстрее, чем удержание платящих.

In [ ]:
retention

In [ ]:
retention_hist

In [ ]:
report = profiles.query(
     # в профилях находим пользователей, привлечённых с 1 по 5 мая
    'datetime.date(2019, 5, 1) <= date <= datetime.date(2019, 5, 7)'
)
print(
    # считаем уникальных пользователей в профилях и складываем размеры когорт
    'Общее количество новых пользователей: {} {}'.format(
        len(report['user_id'].unique()),
        0 # retention['cohort_size'].sum()
    )
)

In [ ]:
report = profiles.query(
    # в профилях находим платящих пользователей, привлечённых с 1 по 5 мая
    'datetime.date(2019, 5, 1) <= date <= datetime.date(2019, 5, 7) and payer == True'
)
print(
    # считаем уникальных платящих пользователей в профилях 
    # и складываем размеры платящих когорт
    'Общее количество новых покупателей: {} {}'.format(
        len(report['user_id'].unique()),
        0 #retention.query('payer == True')['cohort_size'].sum(),
    )  
)

Проверяем поведение графиков. Эталонные графики:

![](https://pictures.s3.yandex.net/resources/graph_5_1620501686.png)

Фактические:

In [ ]:
retention.T.plot(grid=True, xticks=list(retention.columns.values), figsize=(15, 5))
plt.title('Удержание с разбивкой по покупкам')
plt.xlabel('Лайфтайм')
plt.ylabel('Доля от всей когорты')
plt.show()

Похоже, всё в порядке:
- кривая удержания платящих выше;
- обе кривые снижаются.

#### Конверсия Conversion Rate

При расчёте конверсии удостоверьтесь, что:
- сумма когорт равна общему числу новых клиентов за выбранный период;
- количество новых покупателей равно числу клиентов, умноженному на конверсию;
- кривая конверсии плавно растёт от нуля к лимиту единицы, без единого снижения;
- не превышает единицу.

In [ ]:
profiles

In [ ]:
conversion

In [ ]:
conversion_history

In [ ]:
conversion_history['cohort_size'].sum() == profiles.query(
    'datetime.date(2019, 5, 1) <= date <= datetime.date(2019, 5, 8)'
)['user_id'].nunique()

![](https://pictures.s3.yandex.net/resources/graph_7_1620501905.png)

In [ ]:
report = conversion.drop(columns={'cohort_size'})
report.T.plot(grid=True, xticks=list(report.columns.values), figsize=(12, 5))
plt.title('Кривая конверсии')
plt.xlabel('Лайфтайм')
plt.ylabel('Доля в размере когорты')
plt.show()

In [ ]:
# только значения больше единицы

report[report > 1].fillna('')  # скрывает значения ниже нуля

#### Полная ценность пользователя LTV

- Сумма размеров когорт равна общему числу новых клиентов в изучаемый период.
- Кривая LTV плавно растёт от нуля с возможным пересечением единицы.
- Кривая LTV не снижается.
- Общая стоимость покупок новых клиентов равна максимальному LTV, умноженному на число новых клиентов.

![](https://pictures.s3.yandex.net/resources/graph_6_1620502104.png)

Как и конверсия, LTV повышается по экспоненте.

#### Стоимость привлечения клиента CAC

- CAC из таблицы ROI, умноженный на размер когорты, равен сумме рекламных трат за изучаемый период.

#### Возврат на инвестиции, или ROI

Если вы проверили LTV и САС, то за ROI можно не волноваться: раз все компоненты верны, верен и результат.

Единственное, на что стоит обратить внимание, — порядок значений.

Реалистичный ROI находится в пределах от 0 до 3. Ведь вполне может быть, что затраты на привлечение не окупились — тогда ROI меньше 100%; или наоборот, окупились с лихвой — тогда ROI больше 100%. Если же затраты на привлечение окупились более чем на 300%, то перед вами либо маркетинговое чудо, либо ошибка в расчётах.